# Angle recalculations

The angular deviation between two vectors will be reconstructed using two methods:
   - The acute angular method: Calculated by applying cosine.
   - The obtuse angular method: Calculated by applying atan2.
   
 

In [37]:
import os
import pandas as pd
import numpy as np
import warnings
import math
warnings.simplefilter(action='ignore', category=FutureWarning)

### Load corrected integrated Pointing task

Data comes as a result of script `02_ParticipantsDataReconstruction`

In [38]:
CoordinatesIntegrated = pd.read_csv('/Users/tracysanchezpacheco/Documents/Resources/PointingTask_Integrated.csv')
CoordinatesIntegrated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9408 entries, 0 to 9407
Data columns (total 33 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Unnamed: 0                     9408 non-null   int64  
 1   SubjectID                      9408 non-null   int64  
 2   ParticipantPosition_x          9408 non-null   float64
 3   ParticipantPosition_z          9408 non-null   float64
 4   PointerPosition_x              9408 non-null   float64
 5   PointerPosition_z              9408 non-null   float64
 6   PointerDirection_x             9408 non-null   float64
 7   PointerDirection_z             9408 non-null   float64
 8   BuildingCenterWorld_z          9408 non-null   float64
 9   BuildingCenterWorld_x          9408 non-null   float64
 10  DistanceToParticipant          9408 non-null   float64
 11  PointingTaskStartingLocations  9408 non-null   int64  
 12  AgentID                        9408 non-null   i

## Creation of vectors

In [39]:
CoordinatesIntegrated['vector_PartPos'] = list(zip(CoordinatesIntegrated.ParticipantPosition_x, CoordinatesIntegrated.ParticipantPosition_z))
CoordinatesIntegrated['vector_BuildingPos'] = list(zip(CoordinatesIntegrated['BuildingCenterWorld_x'], CoordinatesIntegrated['BuildingCenterWorld_z']))
CoordinatesIntegrated['vector_PointerPos'] =list(zip(CoordinatesIntegrated.PointerDirection_x, CoordinatesIntegrated.PointerDirection_z))

In [40]:
PartList = list(CoordinatesIntegrated['vector_PartPos'])
BuiltList = list(CoordinatesIntegrated['vector_BuildingPos'])
PointList = list(CoordinatesIntegrated['vector_PointerPos'])

In [41]:
### Calculating direction vector from Participant Position (a) to Target Building Position (b): b- **  component form is terminal minus initial
CoordinatesIntegrated['Vector_Participant-Building'] = [tuple(x-y for x, y in zip(tup1, tup2)) for tup1, tup2 in zip(BuiltList, PartList)]
### Calculating direction vector from Participant Position (a) to pointer (b): a-b
CoordinatesIntegrated['Vector_Participant-Pointer'] = [tuple(x-y for x, y in zip(tup1, tup2)) for tup1, tup2 in zip(PointList, PartList)] # addition
CoordinatesIntegrated.iloc[:,-15:-8].head(5)

,Trials,TrialNumber,StartPointID.1,ID_for_StartingPosition,ImageName,TimeDelta,RT
0,9,1,1,1,05_CmANo,0 days 00:00:25.632290816,25.632291
1,8,49,5,1,05_CmA,0 days 00:00:09.823777024,9.823777
2,8,173,15,5,05_CmA,0 days 00:00:02.202241280,2.202241
3,9,176,15,8,05_CmANo,0 days 00:00:06.977730048,6.977730
4,9,262,22,10,05_CmANo,0 days 00:00:06.163793408,6.163794


In [42]:
CoordinatesIntegrated.groupby(['BuildingCenterWorld_z','BuildingCenterWorld_x',  'ImageName']).count()

Unnamed: 0  SubjectID  \
BuildingCenterWorld_z BuildingCenterWorld_x ImageName                          
-255.873932           -3.336735             23_CmA            102        102   
                                            23_CmANo           77         77   
-205.836075           -42.972923            48_Sa              88         88   
                                            48_SaNo            90         90   
-204.096573            96.337585            40_Sa              82         82   
...                                                           ...        ...   
 226.580154           -72.436798            18_CmANo           74         74   
 234.419739            197.370117           03_CmA             68         68   
                                            03_CmANo           91         91   
 288.053894           -234.506821           56_Sa              80         80   
                                            56_SaNo            88         88   

                                                       ParticipantPosition_x  \
BuildingCenterWorld_z BuildingCenterWorld_x ImageName                          
-255.873932           -3.336735             23_CmA                       102   
                                            23_CmANo                      77   
-205.836075           -42.972923            48_Sa                         88   
                                            48_SaNo                       90   
-204.096573            96.337585            40_Sa                         82   
...                                                                      ...   
 226.580154           -72.436798            18_CmANo                      74   
 234.419739            197.370117           03_CmA                        68   
                                            03_CmANo                      91   
 288.053894           -234.506821           56_Sa                         80   
                                            56_SaNo                       88   

                                                       ParticipantPosition_z  \
BuildingCenterWorld_z BuildingCenterWorld_x ImageName                          
-255.873932           -3.336735             23_CmA                       102   
                                            23_CmANo                      77   
-205.836075           -42.972923            48_Sa                         88   
                                            48_SaNo                       90   
-204.096573            96.337585            40_Sa                         82   
...                                                                      ...   
 226.580154           -72.436798            18_CmANo                      74   
 234.419739            197.370117           03_CmA                        68   
                                            03_CmANo                      91   
 288.053894           -234.506821           56_Sa                         80   
                                            56_SaNo                       88   

                                                       PointerPosition_x  \
BuildingCenterWorld_z BuildingCenterWorld_x ImageName                      
-255.873932           -3.336735             23_CmA                   102   
                                            23_CmANo                  77   
-205.836075           -42.972923            48_Sa                     88   
                                            48_SaNo                   90   
-204.096573            96.337585            40_Sa                     82   
...                                                                  ...   
 226.580154           -72.436798            18_CmANo                  74   
 234.419739            197.370117           03_CmA                    68   
                                            03_CmANo                  91   
 288.053894           -234.506821           56_Sa                     80   
                    

## Angle functions acute method

In [43]:
def DotTuple(v1,v2):
    """Returns the dot product of two vectors of two elements each
Parameters
-------------
    v1: tuple of two elements
    v2: tuple of two elements
returns
----------
    list of dot product of tuples
"""
    dotList = []
    dot = [sum(x*y for x, y in zip(tup1, tup2))for tup1, tup2 in zip(v1, v2)]
    dotList.append(dot)
    flatlist = [item for sublist in dotList for item in sublist]
    return flatlist

In [44]:
def Magnitud(v1):
    """Returns the magnitute of a vector
Parameters
-------------
    v1: tuple of two elements
returns
----------
    list with magnitude of vectors
"""
    v1 = [x ** 2 + y ** 2 for x,y in v1]
    squared = [math.sqrt(value) for value in v1]
    return(squared)

In [45]:
CoordinatesIntegrated['building-Participant_x'] = CoordinatesIntegrated['BuildingCenterWorld_x'] -  CoordinatesIntegrated['ParticipantPosition_x']
CoordinatesIntegrated['building-Participant_z'] = CoordinatesIntegrated['BuildingCenterWorld_z'] - CoordinatesIntegrated['ParticipantPosition_z']

In [46]:
# Creating the individual elements of the arc cos formula and inserting them on the dataframe
CoordinatesIntegrated['Dot_Pointer-Building-Participant'] = DotTuple(CoordinatesIntegrated['Vector_Participant-Pointer'],CoordinatesIntegrated['Vector_Participant-Building'])
CoordinatesIntegrated['Magnitud_Participant-Pointer'] = Magnitud(CoordinatesIntegrated['Vector_Participant-Pointer'])
CoordinatesIntegrated['Magnitud_Participant-Building'] = Magnitud(CoordinatesIntegrated['Vector_Participant-Building'])

In [47]:
# Calculating degrees using the arc cos formula and inserting them on the dataframe
my_angle = CoordinatesIntegrated['Dot_Pointer-Building-Participant']/ (CoordinatesIntegrated['Magnitud_Participant-Pointer'] * CoordinatesIntegrated['Magnitud_Participant-Building'])
CoordinatesIntegrated['radians'] =  [math.acos(value) for value in my_angle]
CoordinatesIntegrated['degrees'] = [np.rad2deg(value) for value in CoordinatesIntegrated['radians']]
CoordinatesIntegrated['degrees'].describe()

count    9408.000000
mean       42.838051
std        38.198586
min         0.006036
25%        14.470790
50%        31.088278
75%        59.811312
max       179.736428
Name: degrees, dtype: float64

## Angle functions obtuse method

In [48]:
def signedAngle(v1, v2):
    """" Returns the subtraction of arc tangent of vector 2 minus vector 1
Parameters
-------------
v1: tuple of values that contains the coordinates of your reference initial vector,
    in our case the perfect angle between position and target (remember to order them y,x)
v2: tuple of values that contains the coordinates of your vector of contrast,
    in our case the real reply from the participants (remember to order them y,x)
returns
----------

"""
    angle = math.atan2(v2[1], v2[0]) - math.atan2(v1[1], v1[0])
    return(np.rad2deg(angle))

In [49]:
signedAngles = []

for count, x in enumerate(CoordinatesIntegrated['PointerDirection_z']):
    signedAngles = signedAngles + [signedAngle([ CoordinatesIntegrated['PointerDirection_z'][count], CoordinatesIntegrated['PointerDirection_x'][count]], [CoordinatesIntegrated['building-Participant_z'][count]
       , CoordinatesIntegrated['building-Participant_x'][count]])]

# Account for angle deviations > 180°
for count, angle in enumerate(signedAngles):
    if angle < 0:
        signedAngles[count] = angle + 360
CoordinatesIntegrated.insert(11, "SignedAngle360", signedAngles, True)

for count, angle in enumerate(signedAngles):
    if angle > 180:
        signedAngles[count] = angle - 360
    elif angle < -180:
        signedAngles[count] = angle + 360
CoordinatesIntegrated.insert(11, "SignedAngle-+180", signedAngles, True)
CoordinatesIntegrated['SignedAngle-+180'].describe()

count    9408.000000
mean       -6.209709
std        65.613022
min      -179.881808
25%       -40.558605
50%        -5.681134
75%        24.726616
max       179.983386
Name: SignedAngle-+180, dtype: float64

In [50]:
PointerZ = CoordinatesIntegrated.loc[:,'PointerDirection_z'].values
PointerX = CoordinatesIntegrated.loc[:,'PointerDirection_x'].values
BuildingParticipantZ = CoordinatesIntegrated.loc[:,'building-Participant_z'].values
BuldingParticipantX= CoordinatesIntegrated.loc[:,'building-Participant_x'].values

In [51]:
CoordinatesIntegrated['atanPointer'] = [math.atan2(z, x) for z, x in zip(PointerZ , PointerX)]
CoordinatesIntegrated['atanBuild-Part'] = [math.atan2(z, x) for z, x in zip(BuildingParticipantZ , BuldingParticipantX)]
CoordinatesIntegrated['atanSubs'] = CoordinatesIntegrated['atanPointer'] - CoordinatesIntegrated['atanBuild-Part']
CoordinatesIntegrated['rad2degree']= [np.rad2deg(value) for value in CoordinatesIntegrated['atanSubs']]
signedAngles = CoordinatesIntegrated['rad2degree'].values

In [52]:
for count, angle in enumerate(signedAngles):
    if angle > 180:
        signedAngles[count] = angle - 360
    elif angle < -180:
        signedAngles[count] = angle + 360
CoordinatesIntegrated.insert(11, "SignedAngle-+1802", signedAngles, True)

In [53]:
CoordinatesIntegrated['AbsolutError'] = abs(CoordinatesIntegrated['rad2degree'])
CoordinatesIntegrated[['AbsolutError']].describe()

,AbsolutError
count,9408.000000
mean,48.265339
std,44.882953
min,0.009861
25%,12.452814
50%,33.523792
75%,71.989249
max,179.983386


## Identifying extreme values and flagging first trials

# Flagging the absolute values that transcend 1.5 times and 3 times the inter-quartile range

In [54]:
def find_iqr(x):
    """"
    Returns inter-quartile range of a numeric (i.e, integer or float)
Parameters
-------------
    x: Numeric (i.e, integer or float) variable one wants to calculate interquartile range from
returns
----------
    float that contains interquartile range
    """
    return np.subtract(*np.percentile(x, [75, 25]))


In [55]:
# Create and empty dataframe that will countain interquartile range
inter_quartile_range = pd.DataFrame(columns=['SubjectID', 'IQR'])

#Loop through all subjects and calculate personal IQR
for i, subject in enumerate(CoordinatesIntegrated.SubjectID.unique()):
    inter_quartile_range.at[i, 'SubjectID'] = subject
    inter_quartile_range.at[i, 'IQR'] = find_iqr(CoordinatesIntegrated[CoordinatesIntegrated.SubjectID == subject]['AbsolutError'])

inter_quartile_range['IQRlimit'] = inter_quartile_range['IQR'] * 1.5
inter_quartile_range['IQRlimit2'] = inter_quartile_range['IQR'] * 3

inter_quartile_range.sort_values('IQR')

,SubjectID,IQR,IQRlimit,IQRlimit2
11,4917,15.933994,23.90099,47.801981
9,4176,22.212112,33.318168,66.636336
18,7412,22.267043,33.400564,66.801128
0,479,25.855071,38.782607,77.565214
2,2258,31.996184,47.994275,95.988551
26,9586,41.605113,62.40767,124.81534
15,6642,41.924538,62.886807,125.773614
5,3246,43.714881,65.572322,131.144644
3,2361,45.523749,68.285624,136.571248
8,3976,48.755161,73.132742,146.265483


In [56]:
# Merged finished data base with inter-quartile range base
interQ = pd.merge(inter_quartile_range,CoordinatesIntegrated, on='SubjectID')
interQ.head(50)

,SubjectID,IQR,IQRlimit,IQRlimit2,Unnamed: 0,ParticipantPosition_x,ParticipantPosition_z,PointerPosition_x,PointerPosition_z,PointerDirection_x,...,Dot_Pointer-Building-Participant,Magnitud_Participant-Pointer,Magnitud_Participant-Building,radians,degrees,atanPointer,atanBuild-Part,atanSubs,rad2degree,AbsolutError
0,479,25.855071,38.782607,77.565214,0,174.565613,140.809692,174.247833,139.919708,-0.488963,...,66776.007572,225.203300,297.572725,0.084366,4.833836,-2.084065,-2.545554,0.461489,26.441377,26.441377
1,479,25.855071,38.782607,77.565214,2,-142.220413,-185.136780,-142.066452,-184.297974,0.285370,...,39619.954136,234.388836,173.846204,0.235808,13.510811,1.280485,1.153080,0.127405,7.299795,7.299795
2,479,25.855071,38.782607,77.565214,4,-54.895859,235.641327,-55.132767,235.065857,-0.440328,...,61029.230009,242.723846,262.417916,0.290340,16.635255,-2.027988,-1.634858,-0.393129,-22.524660,22.524660
3,479,25.855071,38.782607,77.565214,6,-54.882858,235.573914,-55.164627,235.090149,-0.548326,...,60981.321518,242.572809,262.351473,0.290037,16.617918,-2.151675,-1.634924,-0.516751,-29.607650,29.607650
4,479,25.855071,38.782607,77.565214,8,-101.070389,122.244217,-101.253830,121.961014,-0.362058,...,21247.075678,159.101590,151.360317,0.490086,28.079838,-1.943147,-1.375484,-0.567663,-32.524715,32.524715
5,479,25.855071,38.782607,77.565214,10,-228.852768,189.914307,-228.790115,189.486603,0.207335,...,77260.140114,298.174508,267.245917,0.247376,14.173609,-1.361691,-0.942135,-0.419556,-24.038798,24.038798
6,479,25.855071,38.782607,77.565214,12,2.501694,41.494225,1.941864,41.325203,-0.939452,...,3088.702667,41.973191,100.463456,0.748838,42.905261,-2.796541,-2.401711,-0.394830,-22.622087,22.622087
7,479,25.855071,38.782607,77.565214,14,-243.134811,39.133606,-242.769119,39.015350,0.913717,...,44423.883519,247.229747,183.480110,0.203699,11.671115,-0.412150,-0.364293,-0.047857,-2.742014,2.742014
8,479,25.855071,38.782607,77.565214,16,-242.994522,39.181503,-242.767929,39.021866,0.920065,...,44370.881627,247.103123,183.366117,0.203990,11.687753,-0.397708,-0.364810,-0.032898,-1.884937,1.884937
9,479,25.855071,38.782607,77.565214,18,-242.984116,39.292313,-242.606277,39.034195,0.919398,...,44378.345271,247.110581,183.395960,0.204109,11.694599,-0.401528,-0.365394,-0.036133,-2.070286,2.070286


In [57]:
## Add flag that's true if the puntuaction should stay false if it should go
interQ['Flag'] =  [limit > absolut  for absolut,limit in zip(interQ['AbsolutError'],interQ['IQRlimit'])]
interQ['Flag2'] =  [limit > absolut  for absolut,limit in zip(interQ['AbsolutError'],interQ['IQRlimit2'])]
interQ.head(50)

,SubjectID,IQR,IQRlimit,IQRlimit2,Unnamed: 0,ParticipantPosition_x,ParticipantPosition_z,PointerPosition_x,PointerPosition_z,PointerDirection_x,...,Magnitud_Participant-Building,radians,degrees,atanPointer,atanBuild-Part,atanSubs,rad2degree,AbsolutError,Flag,Flag2
0,479,25.855071,38.782607,77.565214,0,174.565613,140.809692,174.247833,139.919708,-0.488963,...,297.572725,0.084366,4.833836,-2.084065,-2.545554,0.461489,26.441377,26.441377,True,True
1,479,25.855071,38.782607,77.565214,2,-142.220413,-185.136780,-142.066452,-184.297974,0.285370,...,173.846204,0.235808,13.510811,1.280485,1.153080,0.127405,7.299795,7.299795,True,True
2,479,25.855071,38.782607,77.565214,4,-54.895859,235.641327,-55.132767,235.065857,-0.440328,...,262.417916,0.290340,16.635255,-2.027988,-1.634858,-0.393129,-22.524660,22.524660,True,True
3,479,25.855071,38.782607,77.565214,6,-54.882858,235.573914,-55.164627,235.090149,-0.548326,...,262.351473,0.290037,16.617918,-2.151675,-1.634924,-0.516751,-29.607650,29.607650,True,True
4,479,25.855071,38.782607,77.565214,8,-101.070389,122.244217,-101.253830,121.961014,-0.362058,...,151.360317,0.490086,28.079838,-1.943147,-1.375484,-0.567663,-32.524715,32.524715,True,True
5,479,25.855071,38.782607,77.565214,10,-228.852768,189.914307,-228.790115,189.486603,0.207335,...,267.245917,0.247376,14.173609,-1.361691,-0.942135,-0.419556,-24.038798,24.038798,True,True
6,479,25.855071,38.782607,77.565214,12,2.501694,41.494225,1.941864,41.325203,-0.939452,...,100.463456,0.748838,42.905261,-2.796541,-2.401711,-0.394830,-22.622087,22.622087,True,True
7,479,25.855071,38.782607,77.565214,14,-243.134811,39.133606,-242.769119,39.015350,0.913717,...,183.480110,0.203699,11.671115,-0.412150,-0.364293,-0.047857,-2.742014,2.742014,True,True
8,479,25.855071,38.782607,77.565214,16,-242.994522,39.181503,-242.767929,39.021866,0.920065,...,183.366117,0.203990,11.687753,-0.397708,-0.364810,-0.032898,-1.884937,1.884937,True,True
9,479,25.855071,38.782607,77.565214,18,-242.984116,39.292313,-242.606277,39.034195,0.919398,...,183.395960,0.204109,11.694599,-0.401528,-0.365394,-0.036133,-2.070286,2.070286,True,True


In [58]:
interQ.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9408 entries, 0 to 9407
Data columns (total 58 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   SubjectID                         9408 non-null   object 
 1   IQR                               9408 non-null   object 
 2   IQRlimit                          9408 non-null   object 
 3   IQRlimit2                         9408 non-null   object 
 4   Unnamed: 0                        9408 non-null   int64  
 5   ParticipantPosition_x             9408 non-null   float64
 6   ParticipantPosition_z             9408 non-null   float64
 7   PointerPosition_x                 9408 non-null   float64
 8   PointerPosition_z                 9408 non-null   float64
 9   PointerDirection_x                9408 non-null   float64
 10  PointerDirection_z                9408 non-null   float64
 11  BuildingCenterWorld_z             9408 non-null   float64
 12  Buildi

In [59]:
interQ.Flag.value_counts()

True     7534
False    1874
Name: Flag, dtype: int64

In [60]:
interQ.Flag2.value_counts()

True     9146
False     262
Name: Flag2, dtype: int64

In [61]:
CoordinatesIntegrated.columns

Index(['Unnamed: 0', 'SubjectID', 'ParticipantPosition_x',
       'ParticipantPosition_z', 'PointerPosition_x', 'PointerPosition_z',
       'PointerDirection_x', 'PointerDirection_z', 'BuildingCenterWorld_z',
       'BuildingCenterWorld_x', 'DistanceToParticipant', 'SignedAngle-+1802',
       'SignedAngle-+180', 'SignedAngle360', 'PointingTaskStartingLocations',
       'AgentID', 'Name', 'StartPointID', 'PointingTargetCondition', 'Context',
       'Agent', 'avatarID', 'AvatarPresence', 'AvatarPresenceCategory',
       'meaningful', 'meaningfulBuilding', 'Trials', 'TrialNumber',
       'StartPointID.1', 'ID_for_StartingPosition', 'ImageName', 'TimeDelta',
       'RT', 'TimeOut', 'TimeStampBegin', 'TimeStampEnd', 'vector_PartPos',
       'vector_BuildingPos', 'vector_PointerPos',
       'Vector_Participant-Building', 'Vector_Participant-Pointer',
       'building-Participant_x', 'building-Participant_z',
       'Dot_Pointer-Building-Participant', 'Magnitud_Participant-Pointer',
       'M

## First pointing trial

In our design the participants have to point to targets from 28 starting locations, they switch positions every 12 trials. It would be sensible to assume that the first trial in each location can count more variance of error since they are starting to familiarize themselves with the environment.

In [62]:
# Flag the first trial of every starting position.
# This has the limitation that it wrongly flags for the last trial of first pointing position
interQ['Removefirst'] = [not (value % 12 == 0) for value in interQ['TrialNumber']]
# The not operator it's used in this case so that all trials that stay are true and the ones to be eliminated are false
interQ[['TrialNumber', 'Removefirst']].head(13)

,TrialNumber,Removefirst
0,1,True
1,49,True
2,173,True
3,176,True
4,262,True
5,270,True
6,277,True
7,313,True
8,314,True
9,317,True


In [63]:
# Here we correct for the mistake the previous method created (i.e., deflag the 12th trial)
interQ.loc[interQ['TrialNumber'] == 12, 'Removefirst'] = True
# And we flag all first trials
interQ.loc[interQ['TrialNumber'] == 1, 'Removefirst'] = False

In [64]:
# Proof of concept: Here we retain only the flagged trials and check if the correct Trial Ids have been identified
TeoryFirst = interQ[interQ['Removefirst'] == False].copy()
check = pd.DataFrame(TeoryFirst['TrialNumber'].unique())
check.describe()

,0
count,28.000000
mean,173.607143
std,99.399806
min,1.000000
25%,93.000000
50%,174.000000
75%,255.000000
max,336.000000


In [65]:
# Added cell to set Working Directory to your location
os.chdir('/Users/tracysanchezpacheco/Documents/Resources')
os.getcwd()
interQ.to_csv('PointingTask_IQR_Final.csv')

In [66]:
interQ.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9408 entries, 0 to 9407
Data columns (total 59 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   SubjectID                         9408 non-null   object 
 1   IQR                               9408 non-null   object 
 2   IQRlimit                          9408 non-null   object 
 3   IQRlimit2                         9408 non-null   object 
 4   Unnamed: 0                        9408 non-null   int64  
 5   ParticipantPosition_x             9408 non-null   float64
 6   ParticipantPosition_z             9408 non-null   float64
 7   PointerPosition_x                 9408 non-null   float64
 8   PointerPosition_z                 9408 non-null   float64
 9   PointerDirection_x                9408 non-null   float64
 10  PointerDirection_z                9408 non-null   float64
 11  BuildingCenterWorld_z             9408 non-null   float64
 12  Buildi

In [67]:
#Elimininating trails were the participant ran out of time
Clean = interQ[interQ['TimeOut']== False]

In [68]:
PointingSum = pd.DataFrame(Clean[['SubjectID', 'AbsolutError', 'SignedAngle-+180', 'IQR', 'RT',  'DistanceToParticipant', 'PointingTaskStartingLocations',  'TrialNumber', 'StartPointID', 'ID_for_StartingPosition','avatarID', 'AvatarPresenceCategory', 'Context', 'meaningfulBuilding',  'ImageName']].values)

In [69]:
PointingSum.columns = ['ID', 'AbsolutError', 'SignedAngle-+180', 'IQR', 'RT',  'DistanceToParticipant', 'PointingTaskStartingLocations',  'TrialNumber', 'StartPointID', 'ID_for_StartingPosition','avatarID', 'AvatarPresenceCategory', 'Context', 'meaningfulBuilding',  'ImageName']
PointingSum.head()

,ID,AbsolutError,SignedAngle-+180,IQR,RT,DistanceToParticipant,PointingTaskStartingLocations,TrialNumber,StartPointID,ID_for_StartingPosition,avatarID,AvatarPresenceCategory,Context,meaningfulBuilding,ImageName
0,479,26.441377,26.441377,25.855071,25.632291,297.913635,7,1,1,1,5,Omitted,True,Meaningful,05_CmANo
1,479,7.299795,7.299795,25.855071,9.823777,173.046249,22,49,5,1,5,Present,True,Meaningful,05_CmA
2,479,22.52466,-22.52466,25.855071,2.202241,263.186127,18,173,15,5,5,Present,True,Meaningful,05_CmA
3,479,29.60765,-29.60765,25.855071,6.97773,263.186127,18,176,15,8,5,Omitted,True,Meaningful,05_CmANo
4,479,32.524715,-32.524715,25.855071,6.163794,152.117493,19,262,22,10,5,Omitted,True,Meaningful,05_CmANo


In [70]:
os.chdir('/Users/tracysanchezpacheco/Documents/Resources')
PointingSum.to_csv('PointingSummary.csv')
Clean.to_csv('FULL_Pointing.csv')

In [71]:
PointingSum[['ImageName', 'avatarID']]

,ImageName,avatarID
0,05_CmANo,5
1,05_CmA,5
2,05_CmA,5
3,05_CmANo,5
4,05_CmANo,5
...,...,...
9379,36_SaNo,36
9380,36_Sa,36
9381,36_SaNo,36
9382,36_SaNo,36
